# first test

fdafsd

fdafda


In [48]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import mode
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm, tree

In [49]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
print(train.head())
print(test.head())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
  

In [50]:
test.shape

(418, 11)

In [51]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,NaN,0.000000,0.000000,NaN
50%,1100.500000,3.000000,NaN,0.000000,0.000000,NaN
75%,1204.750000,3.000000,NaN,1.000000,0.000000,NaN
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [52]:
train.shape

(891, 12)

In [53]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [54]:
print train.iloc[124]

PassengerId                            125
Survived                                 0
Pclass                                   1
Name           White, Mr. Percival Wayland
Sex                                   male
Age                                     54
SibSp                                    0
Parch                                    1
Ticket                               35281
Fare                               77.2875
Cabin                                  D26
Embarked                                 S
Name: 124, dtype: object


In [55]:
train = train.drop(['Ticket','Cabin'], axis=1)
print train.iloc[124]


PassengerId                            125
Survived                                 0
Pclass                                   1
Name           White, Mr. Percival Wayland
Sex                                   male
Age                                     54
SibSp                                    0
Parch                                    1
Fare                               77.2875
Embarked                                 S
Name: 124, dtype: object


In [56]:
#cleaning fare column
train.Fare = train.Fare.map(lambda x: np.nan if x==0 else x)
classmeans = train.pivot_table('Fare', index='Pclass', aggfunc='mean')
train.Fare = train[['Fare', 'Pclass']].apply(lambda x: classmeans[x['Pclass']] if pd.isnull(x['Fare']) else x['Fare'], axis=1 )
    
#cleaning the age column
meanAge=np.mean(train.Age)
train.Age=train.Age.fillna(meanAge)

print train

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25          

In [57]:
train.shape

(891, 10)

In [58]:
train = train.dropna()
train.shape

(889, 10)

In [8]:
# Passengers that survived vs passengers that passed away
print(train.Survived.value_counts())


0    549
1    342
Name: Survived, dtype: int64


In [66]:
# Males that survived vs males that passed away
print(train["Survived"][train["Sex"] == 'male'].value_counts())


0    54
1    41
Name: Survived, dtype: int64


In [67]:
# Normalized male survival
print(train["Survived"][train["Sex"] == 'male'].value_counts(normalize = True))


0    0.568421
1    0.431579
Name: Survived, dtype: float64


In [68]:
# Females that survived vs Females that passed away
print(train["Survived"][train["Sex"] == 'female'].value_counts())


1    82
0     6
Name: Survived, dtype: int64


In [69]:
# Normalized female survival
print(train["Survived"][train["Sex"] == 'female'].value_counts(normalize = True))


1    0.931818
0    0.068182
Name: Survived, dtype: float64


In [70]:
# Create the column Child and assign to 'NaN'
train["Child"] = float('NaN')
#print(train["Child"])

In [71]:
# Assign 1 to passengers under 18, 0 to those 18 or older. Print the new column.
train["Child"][train["Age"] < 18] = 1
train["Child"][train["Age"] >= 18] = 0
print(train["Child"])

1      0.0
3      0.0
6      0.0
10     1.0
11     0.0
21     0.0
23     0.0
27     0.0
52     0.0
54     0.0
62     0.0
66     0.0
75     0.0
88     0.0
92     0.0
96     0.0
97     0.0
102    0.0
110    0.0
118    0.0
123    0.0
124    0.0
136    0.0
137    0.0
139    0.0
148    0.0
151    0.0
170    0.0
174    0.0
177    0.0
      ... 
737    0.0
741    0.0
742    0.0
745    0.0
748    0.0
751    1.0
759    0.0
763    0.0
765    0.0
772    0.0
779    0.0
781    1.0
782    0.0
789    0.0
796    0.0
802    1.0
806    0.0
809    0.0
820    0.0
823    0.0
835    0.0
853    1.0
857    0.0
862    0.0
867    0.0
871    0.0
872    0.0
879    0.0
887    0.0
889    0.0
Name: Child, dtype: float64


C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [72]:
# Print normalized Survival Rates for passengers under 18
print(train["Survived"][train["Child"] == 1].value_counts(normalize = True))

1    0.894737
0    0.105263
Name: Survived, dtype: float64


In [74]:
# Convert the male and female groups to integer form
train["Sex"][train["Sex"] == "male"] = 0
train["Sex"][train["Sex"] == "female"] = 1
#print train

     PassengerId  Survived  Pclass  \
1              2         1       1   
3              4         1       1   
6              7         0       1   
10            11         1       3   
11            12         1       1   
21            22         1       2   
23            24         1       1   
27            28         0       1   
52            53         1       1   
54            55         0       1   
62            63         0       1   
66            67         1       2   
75            76         0       3   
88            89         1       1   
92            93         0       1   
96            97         0       1   
97            98         1       1   
102          103         0       1   
110          111         0       1   
118          119         0       1   
123          124         1       2   
124          125         0       1   
136          137         1       1   
137          138         0       1   
139          140         0       1   
148         

C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [75]:
# Impute the Embarked variable
train["Embarked"] = train["Embarked"].fillna("S")

# Convert the Embarked classes to integer form
train["Embarked"][train["Embarked"] == "S"] = 0
train["Embarked"][train["Embarked"] == "C"] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2

# Print the Sex and Embarked columns
print(train["Sex"])
print(train["Embarked"])



1      1
3      1
6      0
10     1
11     1
21     0
23     0
27     0
52     1
54     0
62     0
66     1
75     0
88     1
92     0
96     0
97     0
102    0
110    0
118    0
123    1
124    0
136    1
137    0
139    0
148    0
151    1
170    0
174    0
177    1
      ..
737    0
741    0
742    1
745    0
748    0
751    0
759    1
763    1
765    1
772    1
779    1
781    1
782    0
789    0
796    1
802    0
806    0
809    1
820    1
823    1
835    1
853    1
857    0
862    1
867    0
871    1
872    0
879    1
887    1
889    0
Name: Sex, dtype: object
1      1
3      0
6      0
10     0
11     0
21     0
23     0
27     0
52     1
54     1
62     0
66     0
75     0
88     0
92     0
96     1
97     1
102    0
110    0
118    1
123    0
124    0
136    0
137    0
139    1
148    0
151    0
170    0
174    1
177    1
      ..
737    1
741    0
742    1
745    0
748    0
751    0
759    0
763    0
765    0
772    0
779    0
781    0
782    0
789    1
796    0
802    0
806

C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [77]:
# Create the target and features numpy arrays: target, features_one
target = train["Survived"].values
features_one = train[["Pclass", "Sex", "Age", "Fare"]].values


In [78]:
print features_one

[[1L 1 38.0 71.2833]
 [1L 1 35.0 53.1]
 [1L 0 54.0 51.8625]
 [3L 1 4.0 16.7]
 [1L 1 58.0 26.55]
 [2L 0 34.0 13.0]
 [1L 0 28.0 35.5]
 [1L 0 19.0 263.0]
 [1L 1 49.0 76.7292]
 [1L 0 65.0 61.9792]
 [1L 0 45.0 83.475]
 [2L 1 29.0 10.5]
 [3L 0 25.0 7.65]
 [1L 1 23.0 263.0]
 [1L 0 46.0 61.175]
 [1L 0 71.0 34.6542]
 [1L 0 23.0 63.3583]
 [1L 0 21.0 77.2875]
 [1L 0 47.0 52.0]
 [1L 0 24.0 247.5208]
 [2L 1 32.5 13.0]
 [1L 0 54.0 77.2875]
 [1L 1 19.0 26.2833]
 [1L 0 37.0 53.1]
 [1L 0 24.0 79.2]
 [2L 0 36.5 26.0]
 [1L 1 22.0 66.6]
 [1L 0 61.0 33.5]
 [1L 0 56.0 30.6958]
 [1L 1 50.0 28.7125]
 [2L 0 1.0 39.0]
 [2L 0 3.0 26.0]
 [1L 1 44.0 27.7208]
 [1L 1 58.0 146.5208]
 [3L 1 2.0 10.4625]
 [1L 0 40.0 31.0]
 [1L 1 31.0 113.275]
 [1L 1 32.0 76.2917]
 [1L 0 38.0 90.0]
 [1L 1 35.0 83.475]
 [1L 0 44.0 90.0]
 [1L 0 37.0 52.5542]
 [3L 1 29.0 10.4625]
 [1L 0 62.0 26.55]
 [1L 1 30.0 86.5]
 [1L 0 52.0 79.65]
 [1L 0 40.0 0.0]
 [1L 1 58.0 153.4625]
 [1L 1 35.0 135.6333]
 [1L 0 37.0 29.7]
 [1L 1 63.0 77.9583]
 [1L 1

In [79]:
# Fit your first decision tree: my_tree_one
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)

# Look at the importance and score of the included features
print(my_tree_one.feature_importances_)
print(my_tree_one.score(features_one, target))


[ 0.          0.2834689   0.41337348  0.30315762]
1.0


In [80]:
# Impute the missing value with the median
test.Fare[152] = test.Fare.median()


C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [88]:
test = test.dropna()
# Convert the male and female groups to integer form
test["Sex"][test["Sex"] == "male"] = 0
test["Sex"][test["Sex"] == "female"] = 1

# Extract the features from the test set: Pclass, Sex, Age, and Fare.
test_features = test[["Pclass", "Sex", "Age", "Fare"]].values


C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\ronald_haynes\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [89]:

print test_features

[[1L 1 23.0 82.2667]
 [1L 1 47.0 61.175]
 [1L 1 48.0 262.375]
 [1L 1 22.0 61.9792]
 [1L 0 41.0 30.5]
 [1L 0 30.0 57.75]
 [1L 1 45.0 52.5542]
 [1L 0 45.0 29.7]
 [1L 1 60.0 76.2917]
 [1L 0 24.0 60.0]
 [1L 1 28.0 263.0]
 [3L 0 25.0 7.65]
 [1L 1 36.0 262.375]
 [1L 0 13.0 262.375]
 [1L 0 31.0 28.5375]
 [1L 1 60.0 263.0]
 [1L 0 28.5 27.7208]
 [1L 1 35.0 211.5]
 [1L 0 32.5 211.5]
 [1L 1 55.0 25.7]
 [1L 0 67.0 221.7792]
 [1L 1 27.0 52.0]
 [1L 1 76.0 78.85]
 [1L 1 43.0 55.4417]
 [2L 0 18.5 13.0]
 [1L 1 36.0 31.6792]
 [1L 1 63.0 221.7792]
 [3L 1 1.0 16.7]
 [1L 0 36.0 75.2417]
 [1L 1 35.0 57.75]
 [1L 0 53.0 28.5]
 [1L 0 61.0 262.375]
 [1L 1 23.0 83.1583]
 [1L 1 29.0 221.7792]
 [1L 0 42.0 26.55]
 [1L 0 48.0 50.4958]
 [1L 0 54.0 55.4417]
 [2L 1 36.0 39.0]
 [1L 1 64.0 83.1583]
 [1L 0 37.0 83.1583]
 [1L 1 18.0 53.1]
 [1L 1 27.0 247.5208]
 [1L 0 6.0 134.5]
 [1L 0 47.0 227.525]
 [1L 1 33.0 27.7208]
 [1L 0 42.0 42.5]
 [1L 1 50.0 211.5]
 [2L 1 22.0 10.5]
 [1L 0 39.0 71.2833]
 [1L 0 64.0 75.25]
 [1L 1 48.

In [90]:
 # Make your prediction using the test set and print them.
my_prediction = my_tree_one.predict(test_features)
print(my_prediction)


[1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0
 1 1 0 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1 1
 1 1 0 0 0 1 1 1 1 0 0 1 1]


In [91]:
# Create a data frame with two columns: PassengerId & Survived. Survived contains your predictions
PassengerId =np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
print(my_solution)


      Survived
904          1
906          1
916          1
918          1
920          1
926          0
936          1
938          0
940          1
942          1
945          1
949          0
951          1
956          1
960          0
961          1
965          1
966          1
967          1
969          0
973          0
984          1
988          1
992          1
1001         0
1004         1
1006         1
1009         0
1010         0
1014         1
...        ...
1185         0
1197         1
1198         0
1200         0
1206         1
1208         0
1213         0
1214         1
1218         1
1223         0
1227         0
1235         1
1242         1
1247         1
1248         1
1256         1
1263         1
1264         1
1266         1
1270         0
1282         0
1283         0
1287         1
1289         1
1292         1
1296         1
1297         0
1299         0
1303         1
1306         1

[87 rows x 1 columns]


In [93]:
# Write your solution to a csv file with the name my_solution.csv
my_solution.to_csv("my_solution_one.csv", index_label = ["PassengerId"])

